# 03 - Intro Semantic Kernel

## Chat Completion Streaming 


## Environment

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.38.0"
#r "nuget: DotNetEnv, 3.1.1"

using System.IO;
using DotNetEnv;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;

string configurationFile = "../config/config.env";

Env.Load(configurationFile);

string apiKey = Env.GetString("SK_OPENAI_APIKEY");
string endpoint = Env.GetString("SK_OPENAI_ENDPOINT");
string chatCompletionDeployment = Env.GetString("SK_OPENAI_CHATCOMPLETION_DEPLOYMENT_DEFAULT");

Console.WriteLine($"Configuration loaded...");

## Create Kernel


In [2]:
Kernel kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        apiKey: apiKey, 
        endpoint: endpoint, 
        deploymentName: chatCompletionDeployment
    )
    .Build();

Console.WriteLine($"Kernel created...");

## Text Completion Streaming



In [5]:
string grounding = "The Super Sports Championship 2025 was won by the Munich Flying Dolphins with a score of 30-27.";
string prompt = String.Concat(grounding, "\n", "Who won the Super Sports Championship in 2025?");

await foreach (StreamingKernelContent streamingKernelContent in kernel.InvokePromptStreamingAsync(prompt))
{
    if (streamingKernelContent.InnerContent is OpenAI.Chat.StreamingChatCompletionUpdate 
            streamingChatCompletionUpdate)
    {
        if (streamingChatCompletionUpdate.ContentUpdate.Count > 0)
            Console.WriteLine($"{streamingChatCompletionUpdate.ContentUpdate.First().Text}");
    }
        
}       

In [8]:
IChatCompletionService chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

ChatHistory chatHistory = new ChatHistory();
chatHistory.AddUserMessage(prompt);

IAsyncEnumerable<StreamingChatMessageContent> streamingChatMessageContents = chatCompletionService.GetStreamingChatMessageContentsAsync(
    chatHistory: chatHistory
);

await foreach (StreamingChatMessageContent streamingMessageContent in streamingChatMessageContents)
{
    if (!String.IsNullOrEmpty(streamingMessageContent.Content))
        Console.WriteLine($"{streamingMessageContent.Content}");
}